In [3]:
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install pymatgen
!pip install chemparse

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 242 kB 33.1 MB/s            
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 2.6 MB 35.2 MB/s            
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
     |████████████████████████████████| 111 kB 61.9 MB/s            
     |████████████████████████████████| 325 kB 61.4 MB/s            
     |████████████████████████████████| 98 kB 7.3 MB/s             
     |████████████████████████████████| 109 kB 65.0 MB/s            
     |█████████████████████

In [4]:
#Импорт библиотек
import pandas as pd
import numpy as np
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Species
import chemparse as cp

In [5]:
#Записываем все базы данных в переменные
db1 = pd.read_excel('Database_1.xlsx')
db2 = pd.read_excel('Database_2.xlsx')
db3 = pd.read_excel('Database_3.xlsx')
db4 = pd.read_excel('Database_4.xlsx')
db5 = pd.read_excel('Database_5.xlsx')

In [6]:
#Данная функция на основании формулы вещества рассчитывает плотность заряда, которая равна среднему отношению ионного радиуса к степени окисления
def db_fill(data, row):  
    ir_sum = 0
    n = 0
    el = Composition(cp.parse_formula(data.loc[row, 'nanoparticle'])).oxi_state_guesses()
    if len(el) == 0:
        return 0
    elements_list = el[0]
    for i in elements_list.keys():
        os = elements_list[i]
        el = Species(i)
        ir = el.average_ionic_radius / os
        ir_sum += ir
        n += 1
    return(ir_sum/n)

In [7]:
#Переименовали колонки во второй базе данных, а также убрали лишние
db2.rename(columns = {'Nanoparticle': 'nanoparticle', 'Concentration μM': 'concentration_ug/mL',
                      '% Cell viability' : 'viability_%', 'coat': 'coating', 'Diameter (nm)': 'size_nm',
                      'Zeta potential (mV)': 'surface_charge_mV', 'Cells': 'cell_type', 
                      'Cell line (L)/primary cells (P)': 'cell_line', 'Human(H)/Animal(A) cells': 'animal',
                      'Cell morphology': 'cell_morphology', 'Exposure time (h)': 'time_h', 'Test': 'test', 
                      'Type: Organic (O)/inorganic (I)': 'organic'}, inplace = True)

db2 = db2.drop(columns=['Animal?', 'Test indicator', 'Biochemical metric', 'Publication year', 'Particle ID', 'Reference DOI', 
                        'Interference checked (Y/N)', 'Colloidal stability checked (Y/N)', 'Positive control (Y/N)',
                        'Cell age: embryonic (E), Adult (A)', 'Cell-organ/tissue source'])

In [8]:
#Исправили некорректные значения
NP_db2 = {'Zinc oxide': 'ZnO', 'Copper Oxide': 'CuO',  'Iron oxide': 'Fe2O3', 'Hydroxyapatite': 'Ca10(PO4)6(OH)2', 'QDs': 'C'}
db2['nanoparticle'].replace(NP_db2, inplace = True)
db2.loc[21, 'cell_morphology'] = 'Endothelial'

In [9]:
#Добавили молярную массу и электроотрицательность для частиц с формулой
db2['molar_mass'] = [Composition(cp.parse_formula(db2.loc[x, 'nanoparticle'])).weight if x < 2274 else np.nan for x in range(2896)]
db2['electronegativity'] = [Composition(cp.parse_formula(db2.loc[x, 'nanoparticle'])).average_electroneg if x < 2274 else np.nan for x in range(2896)]

In [10]:
#Добавили молярную массу для остальных частиц
db2.loc[db2.nanoparticle == 'Eudragit RL', 'molar_mass'] = 231.29
db2.loc[db2.nanoparticle == 'Dendrimer', 'molar_mass'] = 516.7
db2.loc[db2.nanoparticle == 'PLGA', 'molar_mass'] = 148.11
db2.loc[db2.nanoparticle == 'Polystyrene', 'molar_mass'] = 104.1
db2.loc[db2.nanoparticle == 'Liposomes', 'molar_mass'] = 938.1
db2.loc[db2.nanoparticle == 'Carbon Nanotubes', 'molar_mass'] = 12.01
db2.loc[db2.nanoparticle == 'Carbon NP', 'molar_mass'] = 12.01
db2.loc[db2.nanoparticle == 'Chitosan', 'molar_mass'] = 501.5
db2.loc[db2.nanoparticle == 'SLN', 'molar_mass'] = db2.molar_mass.mean()

In [11]:
#Перевели концентрацию в нужные единицы измерения
db2['concentration_ug/mL'] = db2['concentration_ug/mL'] * db2['molar_mass'] / 1000

In [12]:
#Добавили плотность заряда для частиц с формулой
db2['charge_density'] = [db_fill(db2, x) if x < 2274 else np.nan for x in range(2896)]

In [297]:
#Исправили некорректные значения для четвертой базы данных
db4.loc[[24, 493], 'Material type'] = 'ZnO'
db4.loc[126, 'Material type'] = 'Al2O3'
db4.loc[6, 'Hydro size (nm)'] = 202.8

#Объединили первую и четвертую базы данных
db14 = pd.concat([db1, db4], ignore_index=True, sort=False)

#В объединенной базе данных добавили новые дескрипторы с дефолтными значениями
db14['organic'] = 1
db14['time_h'] = 24
db14['animal'] = 1
db14['coating'] = 'None'
db14['test'] = 'Unknown'
db14['cell_line'] = 'Unknown'
db14['cell_morphology'] = 'Epithelial'

#Переименовали колонки, а также убрали лишние
db14 = db14.drop(['Elements', 'Electronegativity', 'Ionic radius', 'Core size (nm)',
                  'Surface area (m2/g)', 'Number of atoms', 'Molecular weight (g/mol)', 
                  'Topological polar surface area (Å²)', 'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)'], axis=1)
db14 = db14.rename(columns={'Material type': 'nanoparticle', 'Hydro size (nm)': 'size_nm', 
                            'Surface charge (mV)': 'surface_charge_mV', 'Cell type': 'cell_type', 
                            'Exposure dose (ug/mL)': 'concentration_ug/mL', 'Viability (%)': 'viability_%'})

#Исправляем значения
db14.loc[203, 'size_nm'] = 114.7
db14.loc[884, 'size_nm'] = 230.5
db14.loc[[884, 620], 'surface_charge_mV'] = 0.0
db14.loc[647, 'concentration_ug/mL'] = 37.5

#Добавили молярную массу для частиц с формулой
db14['molar_mass'] = [Composition(cp.parse_formula(db14.loc[x, 'nanoparticle'])).weight for x in range(1562)]
db14['electronegativity'] = [Composition(cp.parse_formula(db14.loc[x, 'nanoparticle'])).average_electroneg for x in range(1562)]
db14['charge_density'] = [db_fill(db14, x) for x in range(1562)]

In [298]:
#Переименовали колонки, а также убрали лишние для базы данных 5
db5 = db5.drop(['core_size', 'surf_area', 'toxicity'], axis=1)
db5 = db5.rename(columns={'material': 'nanoparticle', 'hydro_size': 'size_nm', 
                            'surf_charge': 'surface_charge_mV', 'dose': 'concentration_ug/mL', 
                            'cell_species': 'animal', 'viability': 'viability_%', 'time': 'time_h'})

#Добавили новые дескрипторы с дефолтными значениями
db5['coating'] = 'None'
db5['test'] = 'Unknown'
db5['organic'] = 1
db5['cell_morphology'] = 'Epithelial'
db5['cell_line'] = 'Unknown'

#Исправили значения
db5.loc[21, 'surface_charge_mV'] = 42.8

#Добавили молярную массу для частиц с формулой
db5['molar_mass'] = [Composition(cp.parse_formula(db5.loc[x, 'nanoparticle'])).weight for x in range(574)]
db5['electronegativity'] = [Composition(cp.parse_formula(db5.loc[x, 'nanoparticle'])).average_electroneg for x in range(574)]
db5['charge_density'] = [db_fill(db5, x) for x in range(574)]

In [299]:
#SLN - Solid lipid nanoparticles, QD - Quantum dots, PLGA - Polyester, PTFE-PMMA - polytetrafluoroethylene-poly(methyl methacrylate) 
#MWCNT - Multi-walled Carbon Nanotubes, SWCNT - Single-wall carbon nanotubes, EudragitRL - copolymer

#Переименовали колонки, а также убрали лишние для базы данных 3
db3 = db3.rename(columns={'Material': 'nanoparticle', 'Type': 'organic', 'Coat/Functional Group': 'coating', 
                          'Size_in_Water (nm)': 'size_nm', 'Zeta_in_Water (mV)': 'surface_charge_mV', 
                          'Cell_Type': 'cell_type', 'Human_Animal': 'animal', 'Time (hr)': 'time_h', 
                          'Concentration (ug/ml)': 'concentration_ug/mL', 'Test': 'test', 
                          'Cell_Viability (%)': 'viability_%', 'Cell Line_Primary Cell': 'cell_line',
                          'Cell_Morphology': 'cell_morphology', 'Cell_Tissue': 'cell_origin'})

db3 = db3.drop(columns=['No', 'Year', 'DOI', 'Article_ID', 'PDI', 'No_of_Cells (cells/well)', 'Aspect_Ratio', 
                        'Shape', 'Synthesis_Method', 'Surface_Charge', 'Cell_Source','Cell_Age', 'Test_Indicator'])

#Исправили значения
material = {'IronOxide':'Fe2O3', 'Dendrmer':'Dendrimer', 'Ay':'Fe', 'Ce O2': 'CeO2', 'IronOide':'Fe2O3', 'QD': 'C', 'HAP': 'Ca10(PO4)6(OH)2'}
db3['nanoparticle'].replace(material, inplace=True) 
db3.loc[320, 'surface_charge_mV'] = 150 
db3['surface_charge_mV'] = pd.to_numeric(db3['surface_charge_mV'])

#Так как не везде у нас есть размер для воды, то там где его нет он был заменен на размер в растворителе, а если и его не было то просто на размер 
db3.loc[np.isnan(db3["Size_in_Medium (nm)"]), "Size_in_Medium (nm)"] = db3.loc[np.isnan(db3["Size_in_Medium (nm)"]), "Diameter (nm)"]
db3.loc[np.isnan(db3["size_nm"]), "size_nm"] = db3.loc[np.isnan(db3["size_nm"]), "Size_in_Medium (nm)"]
db3 = db3.drop(['Size_in_Medium (nm)', 'Diameter (nm)'], axis=1)

#Аналогично было сделано для заряда
db3.loc[np.isnan(db3["surface_charge_mV"]), "surface_charge_mV"] = db3.loc[np.isnan(db3["surface_charge_mV"]), "Zeta_in_Medium (mV)"]
db3 = db3.drop(['Zeta_in_Medium (mV)'], axis=1)

In [300]:
#Добавили молярную массу для частиц без формулы
db3.loc[db3.nanoparticle == 'Dendrimer', 'molar_mass'] = 516.7
db3.loc[db3.nanoparticle == 'PLGA', 'molar_mass'] = 148.11
db3.loc[db3.nanoparticle == 'Alginate', 'molar_mass'] = 216.12
db3.loc[db3.nanoparticle == 'PTFE-PMMA', 'molar_mass'] = 100.12
db3.loc[db3.nanoparticle == 'Dendrimer', 'molar_mass'] = 516.7
db3.loc[db3.nanoparticle == 'MWCNT', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'Polystyrene', 'molar_mass'] = 104.1
db3.loc[db3.nanoparticle == 'Chitosan', 'molar_mass'] = 501.5
db3.loc[db3.nanoparticle == 'EudragitRL', 'molar_mass'] = 231.29
db3.loc[db3.nanoparticle == 'Carbon', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'SWCNT', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'MWCNT', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'Diamond', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'C70', 'molar_mass'] = 840.7
db3.loc[db3.nanoparticle == 'C60', 'molar_mass'] = 720.66
db3.loc[db3.nanoparticle == 'Graphite', 'molar_mass'] = 12.01
db3.loc[db3.nanoparticle == 'SLN', 'molar_mass'] = db3.molar_mass.mean()

In [301]:
#Добавили молярную массу для частиц с формулой
db3['molar_mass'] = [Composition(cp.parse_formula(db3.loc[x, 'nanoparticle'])).weight 
if np.isnan(db3.loc[x, 'molar_mass']) else db3.loc[x, 'molar_mass'] for x in range(4111)]

In [302]:
#Создали массив частиц для которых нет явной формулы и нельзя найти электроотрицательность и плотность заряда
org = ['Dendrimer', 'PLGA', 'Alginate', 'PTFE-PMMA', 'Dendrimer', 'MWCNT', 'Polystyrene', 'Chitosan', 'EudragitRL', 'Carbon', 'SWCNT',
       'MWCNT', 'Diamond', 'C70', 'C60', 'Graphite', 'SLN']

In [303]:
#Добавили электроотрицательность для частиц с формулой
db3['electronegativity'] = [Composition(cp.parse_formula(db3.loc[x, 'nanoparticle'])).average_electroneg if db3.loc[x, 'nanoparticle'] not in org else np.nan for x in range(4111)]

In [304]:
#Добавили плотность заряда для частиц с формулой
db3['charge_density'] = [db_fill(db3, x) if db3.loc[x, 'nanoparticle'] not in org else np.nan for x in range(4111)]

In [305]:
#Собрали все базы данных в одну
db_final = pd.concat([db14, db5, db2, db3], ignore_index=True, sort=False)

In [306]:
#Убрали лишнюю колонку
db_final = db_final.drop(columns=['cell_origin'])

In [307]:
#Исправили параметр organic_type
organic_type = {'I':1, 'O':0, 'C':1}
db_final['organic'].replace(organic_type, inplace=True) 

In [308]:
#Исправили параметр animal
animal = {'Hamster': 0, 'Mouse': 0, 'A': 0, 'Human': 1, 'H': 1} #в предположении, что 0 - Animal, 1 - Human
db_final['animal'].replace(animal, inplace=True)

In [309]:
#Исправили параметр cell_line
line = {'Unknown': 0, 'L': 1, 'P': 2} #в предположении, что 0 - Unknown, 1 - клеточная линия, 2 - отобрано непосредственно из организма
db_final['cell_line'].replace(line, inplace=True)

In [310]:
#Исправили параметр cell_morphology
db_final['cell_morphology'].fillna('Epithelial', inplace=True)
morphology = {'Epithelial':1, 'Fibroblast':2, 'Endothelial':3,
       'Monocyte/Macrophage':4, 'Lymphoblast':5, 'Monocyte':4,
       'Endothelial-like':3, 'Irregular':0, 'Spindle':7, 'Neuronal':6,
       'Macrophage':4, 'Mesenchymal':8, 'Keratinocyte':9, 'Polygonal':0,
       'Epithelial-like':1, 'Osteoblast':10, 'Mixed':0, 'MulticellularClusters':0}
db_final['cell_morphology'].replace(morphology, inplace=True)

In [311]:
#Прочитали заранее созданные словари соответсвия каждого конкретного значения признака некоторой группе
cell_map = pd.read_csv('cell_map.csv')
coat_map = pd.read_csv('coat_map.csv')
test_map = pd.read_csv('test_map.csv')

In [312]:
#Исправили параметр cell_type
db_final['cell_type'] = db_final['cell_type'].map(cell_map.set_index('name')['type'])
db_final['cell_type'].fillna(0, inplace=True)
type = {'Normal': 0, 'Cancer': 1} #в предположении, что 0 - Normal, 1 - Cancer
db_final['cell_type'].replace(type, inplace=True)

In [313]:
#Исправили параметр coating
# 0	none, 1	amin/aminoacid/protein, 2	alcohol, 3	imide, 4	ether, 5	acid/salt, 6	CnHn, 7	other
db_final['coating'] = db_final['coating'].map(coat_map.set_index('name')['type'])
db_final['coating'].fillna(0, inplace=True)

In [314]:
#Исправили параметр test
#0	Unknown, 1	MTT, 2	XTT, 3	NR, 4	LDH, 5	MTS, 6	Live/Dead, 7	WST­1, 8	CellTiter­Blue, 9	CellTiter­Glo, 10	ATPLite, 11	Alamar Blue, 12	Resazurin, 13	ApoTox­Glo™ Triplex, 14	CytoTox­One™, 15	CVS, 16	EZ4U
db_final['test'] = db_final['test'].map(test_map.set_index('name')['type'])
db_final['test'].fillna(0, inplace=True) 

In [316]:
#Записали исправленную объединенную базу данных в файл
db_final.to_excel("final_data.xlsx")